In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from RGG_Library import RGGBuilder

# ---------------------------
# PARAMETERS
# ---------------------------
space_        = "torus"
perturb_bool  = False
n_instances   = 100
k_values      = np.arange(4.0, 7.01, 0.01)
n_values      = [4**i for i in range(4, 11)]         # 256, 1024, ..., 1048576

print("n values:", n_values)
print(f"k range: {k_values[0]:.2f} → {k_values[-1]:.2f}  ({len(k_values)} steps)")
print(f"Instances per (n,k): {n_instances}")
print(f"Total graphs to build: {len(n_values) * len(k_values) * n_instances:,}\n")

# ---------------------------
# HELPER: return giant component size as fraction of n
# ---------------------------
def giant_component_fraction(n, k, seed):
    try:
        builder = RGGBuilder(
            n=n, k=k, connectivity_regime="sc",
            space=space_, order=4, perturb=perturb_bool, seed=seed
        )
        G = builder.build()
        if G.number_of_nodes() == 0:
            return 0.0
        largest = max(nx.connected_components(G), key=len)
        return len(largest) / n
    except Exception:
        return np.nan

# ---------------------------
# SWEEP
# ---------------------------
# gc_mean[i_n, i_k] = mean giant component fraction over n_instances
gc_mean = np.full((len(n_values), len(k_values)), np.nan)

for i, n in enumerate(n_values):
    print(f"[n={n}]  sweeping k ...", flush=True)
    for j, k in enumerate(k_values):
        fractions = [giant_component_fraction(n, k, seed=s) for s in range(n_instances)]
        gc_mean[i, j] = np.nanmean(fractions)

    print(f"  done. GC range: [{np.nanmin(gc_mean[i]):.3f}, {np.nanmax(gc_mean[i]):.3f}]")

# ---------------------------
# PLOT
# ---------------------------
fig, ax = plt.subplots(figsize=(11, 6))

cmap   = cm.plasma
log_ns = np.log2(np.array(n_values))                 # 8, 10, 12, ..., 20
norm   = plt.Normalize(vmin=log_ns.min(), vmax=log_ns.max())

for i, n in enumerate(n_values):
    color = cmap(norm(log_ns[i]))
    ax.plot(
        k_values, gc_mean[i],
        color=color, linewidth=2,
        label=f"n = {n:,}"
    )

# Reference line at GC = 1.0 (fully connected)
ax.axhline(1.0, color='k', linewidth=1, linestyle='--', alpha=0.4, label='GC = n')

# Colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, fraction=0.03, pad=0.02)
cbar.set_label(r'$\log_2(n)$', fontsize=12)
cbar.set_ticks(log_ns)
cbar.set_ticklabels([f"{n:,}" for n in n_values])

ax.set_xlabel("Mean Degree  $k$", fontsize=13)
ax.set_ylabel("Giant Component Size  (fraction of $n$)", fontsize=13)
ax.set_title(
    f"Giant component transition vs mean degree $k$\n"
    f"({n_instances} instances per point, torus RGG)",
    fontsize=12
)
ax.set_xlim(k_values[0], k_values[-1])
ax.set_ylim(-0.02, 1.05)
ax.legend(fontsize=8, loc='upper left', ncol=2)
ax.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig("giant_component_curves.png", dpi=150)
plt.show()

print("\nDone. Saved to giant_component_curves.png")